In [21]:
!pip install wget

In [22]:
from google.colab import drive
drive.mount('/gdrive')
%cd /gdrive/My\ Drive/software/Unet_Pytorch/Data/

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive/My Drive/software/Unet_Pytorch/Data


In [23]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Wed Oct 14 07:00:21 2020

@author: vicenc
"""


import os
import zipfile
from glob import glob
from typing import List, Tuple

import click
import h5py
import wget
import numpy as np
from PIL import Image,ImageDraw
from scipy.ndimage import gaussian_filter
import math
import json
import cv2


def rotate(origin, point, angle):
    """
    Rotate a point anticlockwise by a given angle around a given origin.
    The angle should be given in radians.
    """
    ox, oy = origin
    px, py = point
    qx = ox + math.cos(angle) * (px - ox) + math.sin(angle) * (py - oy)
    qy = oy - math.sin(angle) * (px - ox) + math.cos(angle) * (py - oy)
    return qx, qy



#####################################################################################################

size_f_x = 577
size_f_y = 417

axis_Width = 5 # Depends on the final image size and size of the objects

rotation_angle_inc = 20 #degrees

targets = ['Sardina','Sardina_inc']

dataname="sardines"
os.chdir('/gdrive/My Drive/software/Unet_Pytorch/Data/' + dataname + '/sardina_train_blaves')

######################################################################################################


In [25]:
JSON_list = glob(os.path.join('*.json'))
JSON_list.sort()

for i, JSON_path in enumerate(JSON_list):
    print(JSON_path)
    img_path = JSON_path.replace('json','jpg')
    image_0 = Image.open(img_path)
    size_o_x, size_o_y = image_0.size
    print(size_o_x, size_o_y)
    rotation_center = (size_o_x/2,size_o_y/2)
     
    for ra in range(0,360,rotation_angle_inc):
        if JSON_path[4]=='4':
          image_r = image_0.rotate(ra+180, resample=Image.BICUBIC, expand=1)
        else:
          image_r = image_0.rotate(ra, resample=Image.BICUBIC, expand=1)
        size_r_x, size_r_y = image_r.size
        image_r = image_r.resize((size_f_x,size_f_y), resample = Image.LANCZOS)
        
        img_path = JSON_path[:-5] + '_r_'+str(ra)+'.jpg'        
        image_r.save(img_path)
        draw_0 = ImageDraw.Draw(image_r)
     
        data = json.load(open(JSON_path))
        mask = np.zeros((size_f_y,size_f_x), dtype=np.uint8)
        mask = Image.fromarray(mask)
        draw = ImageDraw.Draw(mask)
    
        for item in data['shapes']:
            for target in targets:
                if item['label']==target:
                    xy = [tuple(i) for i in item['points']]
                    skeleton = xy
                    skeleton_r = []                    
                    for point in skeleton:
                        p_x=point[0]
                        p_y=-point[1]
                        rc_x = rotation_center[0]
                        rc_y = -rotation_center[1]
                        pd_x=p_x-rc_x
                        pd_y=p_y-rc_y
                        pr_x, pr_y= rotate((0,0),(pd_x,pd_y), -ra*math.pi/180)
                        pr_x = pr_x * size_f_x / size_r_x
                        pr_y = pr_y * size_f_y / size_r_y
                        pr_x =  pr_x + size_f_x / 2
                        pr_y = - pr_y + size_f_y / 2
                        skeleton_r.append((pr_x,pr_y))
                            
                    draw.polygon(xy=skeleton_r, fill='white', outline = 'white')
                    draw_0.polygon(xy=skeleton_r, fill='orange', outline = 'white')
                    
        mask_path = JSON_path[:-5] + '_r_'+str(ra)+'_mask.png'
        mask.save(mask_path)
        img_path = JSON_path[:-5] + '_r_'+str(ra)+'_labelled.jpg'        
        image_r.save(img_path)
    

IMG_4317.json
3024 3024
IMG_4318.json
3024 3024
IMG_4319.json
3024 3024
IMG_4320.json
3024 3024
IMG_4321.json
3024 3024
IMG_4322.json
3024 3024
IMG_4323.json
3024 3024
IMG_4324.json
3024 3024
IMG_4325.json
3024 3024
IMG_4326.json
3024 3024
IMG_4327.json
3024 3024
IMG_4361.json
3024 3024
IMG_4365.json
3024 3024
IMG_4366.json
3024 3024
